In [ ]:
# Analysis

In [1]:
import pandas as pd
import numpy as np
import json
import pickle
from collections import Counter, defaultdict
import matplotlib.pyplot as plt
import seaborn as sns
from bidict import bidict
import sys
import os
from itertools import product
if "/u/solimanz/development/career_prediction_gpu/" not in sys.path:
    sys.path.append("/u/solimanz/development/career_prediction_gpu/")

In [2]:
df_path = "/data/rali7/Tmp/solimanz/data/pickles/excerpt-2017-02-20_reduced.pkl"
base_path = "/data/rali7/Tmp/solimanz/data/"
top550_path = os.path.join(base_path, "datasets", "top550")
reduced7000_path = os.path.join(base_path, "datasets", "reduced7000")
predictions_path = os.path.join(base_path, "model_predictions")

In [4]:
with open(os.path.join(top550_path, "jobid", "data.json"), "r") as f:
    data = json.load(f)

In [5]:
title_id = bidict(data["title_to_id"])

In [6]:
def next_batch(path):
    batch = 0
    name = os.path.basename(path)
    while os.path.exists(os.path.join(path, f"{name}_batch_{batch}.npy")):
        with open(os.path.join(path, f"{name}_batch_{batch}.npy"), "rb") as f:
            matrix = np.load(f)
        batch += 1
        yield matrix

In [15]:
product(['top550', 'reduced7000'], ['jobid', 'bow', 'title_emb'], ['targets', 'predictions', 'seq_lengths'])

In [7]:
jobid_path = os.path.join(predictions_path, "top550", "jobid", "targets")
title_emb_path = os.path.join(predictions_path, "top550", "title_emb", "targets")
bow_path = os.path.join(predictions_path, "top550", "bow", "targets")

In [8]:
jobid_gen = next_batch(jobid_path)
title_emb_gen = next_batch(title_emb_path)
bow_gen = next_batch(bow_path)

In [20]:
[1,2,3] + [2,4,5]

[1, 2, 3, 2, 4, 5]